In [1]:
import numpy as np
import pandas as pd
import glob
import sys,os

In [17]:
from pathlib import Path

In [42]:
sys.path.append('../../')

In [46]:
from utils.bbox.conversation import bbox_coco2voc, bbox_voc2yolo

In [80]:
def bbox_voc2yolo_df(df):
    w =  df['width'].values
    h =  df['height'].values
    w_bbox = df['bx3'].values - df['bx1'].values
    h_bbox = df['bx4'].values - df['bx2'].values
    x_c = df['bx1'].values + w_bbox/2
    y_c = df['bx2'].values + h_bbox/2
    df.loc[:, 'bx1'] = x_c/w
    df.loc[:, 'bx2'] = y_c/h
    df.loc[:, 'bx3'] = w_bbox/w
    df.loc[:, 'bx4'] = h_bbox/h    
    return df

In [4]:
fp_df = pd.read_csv('../fashionpedia/data/fashionpedia6_l1.csv')
fp_df = fp_df[fp_df['label_model']>=0]

In [22]:
hw_df = pd.read_csv('../haowei34k/data/hw34k_bbox_info.csv')
hw_df = hw_df[hw_df['label_model']>=0]

In [94]:
# fp_cat.to_csv('../fashionpedia/data/fashionpedia6_l1_cat.csv', index=False)

In [95]:
# hw_cat.to_csv('../haowei34k/data/hw34k_bbox_info_cat.csv', index=False)

In [96]:
hwpedia_cat = pd.concat([fp_cat, hw_cat], axis=0)

In [98]:
hwpedia_cat.to_csv('./data/hwpedia_bbox.csv', index=False)

In [99]:
gall10k_df = pd.read_csv('./data/tuneDataset/tuneGallery10k_slink_df.csv')

In [100]:
gall10k_df.head()

,fname,label_sp,label_tp,img_tp,img_sp,target
0,20160426020017708,/home/chenyi/workspace/dataset/labels/labels6/...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/dataset/data/haowei34k/...,tuneGallery10k
1,265309ec2dfb86f07dbf30e4664937bf,/home/chenyi/workspace/dataset/labels/labels6/...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/dataset/data/data_fashi...,tuneGallery10k
2,20141226150831480,/home/chenyi/workspace/dataset/labels/labels6/...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/dataset/data/haowei34k/...,tuneGallery10k
3,ef51d854042fa93a94826a27bd69001a,/home/chenyi/workspace/dataset/labels/labels6/...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/dataset/data/data_fashi...,tuneGallery10k
4,20170809192459309,/home/chenyi/workspace/dataset/labels/labels6/...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/yolov5/datasets/tuneGal...,/home/chenyi/workspace/dataset/data/haowei34k/...,tuneGallery10k


In [104]:
hwpedia_cat.head(2)

,source,img_sp,width,height,bx1,bx2,bx3,bx4,label_model,mode,stem
0,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,681.0,1024.0,0.551395,0.898926,0.077827,0.124023,0,train,0002ec21ddb8477e98b2cbb87ea2e269
1,fashionpedia,/home/chenyi/workspace/dataset/data/data_fashi...,681.0,1024.0,0.477974,0.869629,0.063142,0.038086,0,train,0002ec21ddb8477e98b2cbb87ea2e269


In [103]:
hwpedia_cat['stem'] = hwpedia_cat['img_sp'].apply(lambda x: Path(x).stem)

In [111]:
gall_bbox_df = hwpedia_cat[hwpedia_cat['stem'].isin(gall10k_df['fname'])]

In [112]:
len(gall_bbox_df), len(hwpedia_cat)

(34329, 269575)

In [116]:
len(gall10k_df.drop_duplicates('fname'))

10000

In [119]:
hwpedia_cat.to_csv('./data/hwpedia_bbox.csv', index=False)

In [120]:
gall_bbox_df.to_csv('./data/tuneDataset/tuneGallery10k_bbox.csv', index=False)

In [122]:
df_val = gall_bbox_df[gall_bbox_df['mode']=='val']
len(df_val)

4122

In [124]:
df_train = gall_bbox_df[gall_bbox_df['mode']=='train']
len(df_train)

30207

In [125]:
df_train_sample = df_train.sample(25000)

In [126]:
df = pd.concat([df_train_sample, df_val], axis=0)

In [127]:
df.to_csv('./data/tuneDataset/tuneGallery10k_del_bbox.csv', index=False)